In [1]:
import pandas as pd
import numpy as np
from plot_utils import *
from math_utils import *
import pathlib

In [3]:
m_dir = Path('./data/img/mnist/0')
for mnist in m_dir.iterdir():
    M = load_image_as_grayscale_matrix(mnist)
    
    M = upscale_bilinear(M, 100, 100)
    print(np.linalg.matrix_rank(M))
    
    M = upscale_bilinear(M, 200, 200)
    print(np.linalg.matrix_rank(M))
    
    M = upscale_nearest_neighbor(M, 100, 100)
    print(np.linalg.matrix_rank(M))
    
    M = upscale_nearest_neighbor(M, 200, 200)
    print(np.linalg.matrix_rank(M))
    
    print('______________________-')

16
16
16
16
______________________-
16
16
16
16
______________________-
14
14
14
14
______________________-
20
20
20
20
______________________-
14
14
14
14
______________________-
20
20
20
20
______________________-
14
14
14
14
______________________-
19
19
19
19
______________________-
16
16
16
16
______________________-
16
16
16
16
______________________-
14
14
14
14
______________________-
19
19
19
19
______________________-
12
12
12
12
______________________-
18
18
18
18
______________________-
18
18
18
18
______________________-
18
18
18
18
______________________-
19
19
19
19
______________________-
20
20
20
20
______________________-
18
18
18
18
______________________-
14
14
14
14
______________________-
16
16
16
16
______________________-
18
18
18
18
______________________-
20
20
20
20
______________________-
18
18
18
18
______________________-
16
16
16
16
______________________-
19
19
19
19
______________________-
18
18
18
18
______________________-
17
17
17
17
________________

KeyboardInterrupt: 

In [17]:
eig = np.linspace(-1.0, 1, 100)
eig[-1] = 50.0
eig[-1]/eig[0]

np.float64(-50.0)

In [2]:
create_diagonal_matrix_from_eigenvalues([1,2,3,4])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

In [9]:
def thin_qr_factorization(A, threshold=None):
    """
    Perform the Thin QR factorization using Householder reflections.

    Parameters:
        A (numpy.ndarray): Input matrix of size m x n (m >= n).

    Returns:
        R (numpy.ndarray): Square upper triangular matrix of size n x n.
        householder_vectors (list): List of n Householder vectors.
    """
    m, n = A.shape
    A = A.copy()  # To avoid modifying the original matrix
    householder_vectors = []

    if not threshold:
        eps = np.finfo(float).eps
        threshold = eps * np.max(np.abs(A))
    
    for k in range(n):
        # Extract the column vector to be reflected
        x = A[k:m, k]

        # Compute the Householder vector vk
        e1 = np.zeros_like(x)
        e1[0] = np.linalg.norm(x) if x[0] >= 0 else -np.linalg.norm(x)
        vk = x + e1
        if np.max(np.abs(vk)) > threshold:  
            vk /= np.linalg.norm(vk)
        else:
            vk = np.zeros_like(x)

        # Store the Householder vector
        householder_vectors.append(vk)

        # Update the submatrix A[k:m, k:n]
        A[k:m, k:n] -= 2 * np.outer(vk, vk @ A[k:m, k:n])

    # Extract the upper triangular part as R (size n x n)
    R = np.triu(A[:n, :n])
    
    return R, householder_vectors

In [8]:
mat = load_matrices('diagonal', 'diag_identity_10x10',  f'{5}_sketching_g_{10e-16}')

In [13]:
A = mat['A']

In [24]:
mat['V_0']

array([[-0.79518623],
       [-0.29567543],
       [ 1.03540423],
       [ 0.15594223],
       [ 0.739803  ],
       [ 0.46395216],
       [-0.51167344],
       [ 0.43569269],
       [-0.25452119],
       [-0.2591789 ]])

In [10]:
V_0_r, v_h = thin_qr_factorization(mat['V_0'])

In [18]:
v_h

[array([-0.85063684, -0.09773296,  0.34224393,  0.05154536,  0.24453549,
         0.15335538, -0.16912924,  0.14401446, -0.08412978, -0.08566935])]

In [15]:
AQ = apply_householder_transformations(A, v_h)
AQ

array([[-0.44716608],
       [-0.16627051],
       [ 0.58225059],
       [ 0.08769276],
       [ 0.4160218 ],
       [ 0.26089947],
       [-0.28773512],
       [ 0.24500801],
       [-0.14312778],
       [-0.14574701]])

In [17]:
U_t = AQ / V_0_r
U_t

array([[-0.25145997],
       [-0.09350078],
       [ 0.32742357],
       [ 0.04931326],
       [ 0.23394625],
       [ 0.14671456],
       [-0.16180535],
       [ 0.13777813],
       [-0.08048667],
       [-0.08195957]])

In [19]:
U_r, v_h = thin_qr_factorization(U_t)

In [20]:
U_r

array([[0.56234133]])

In [21]:
AQ = apply_householder_transformations(A, v_h)
AQ

array([[-0.44716608],
       [-0.16627051],
       [ 0.58225059],
       [ 0.08769276],
       [ 0.4160218 ],
       [ 0.26089947],
       [-0.28773512],
       [ 0.24500801],
       [-0.14312778],
       [-0.14574701]])

In [22]:
V_t = AQ / U_r
V_t

array([[-0.79518623],
       [-0.29567543],
       [ 1.03540423],
       [ 0.15594223],
       [ 0.739803  ],
       [ 0.46395216],
       [-0.51167344],
       [ 0.43569269],
       [-0.25452119],
       [-0.2591789 ]])